In [1]:
def get_boss_stats():
    with open('input') as f:
        parts = f.read().strip().split()
        return (int(parts[2]), int(parts[4]))

In [2]:
shop = {
    'missile': 53,
    'drain': 73,
    'shield': 113,
    'poison': 173,
    'recharge': 229,
}

In [3]:
def get_cost(moves):
    return sum(shop[move] for move in moves)

In [4]:
from collections import namedtuple

Spell = namedtuple("Spell", ("name", "cost", "damage", "effect_damage", "heal", "effect_time", "reward"))

SPELLS = {
    'missile': Spell("missile", 53, 4, 0, 0, 0, 0),
    'drain': Spell("drain", 73, 2, 0, 2, 0, 0),
    'shield': Spell("shield", 113, 0, 0, 0, 6, 0),
    'poison': Spell("poison", 173, 0, 3, 0, 6, 0),
    'recharge': Spell("recharge", 229, 0, 0, 0, 5, 101),
}

In [5]:
from dataclasses import dataclass

@dataclass
class Player:
    hit_points: int
    mana: int
        
@dataclass
class Boss:
    hit_points: int
    damage: int

In [6]:
class PlayerWins(Exception):
    pass

class BossWins(Exception):
    pass

class EffectActive(Exception):
    pass

class OutOfMoney(Exception):
    pass

In [7]:
class Game:
    def __init__(self, player_stats, boss_stats, hard=False):
        self.player = Player(*player_stats)
        self.boss = Boss(*boss_stats)
        self.effects = {}
        self.hard = hard
        self.player_turn = True
        
    def __next__(self):
        if self.hard and self.player_turn:
            self.player.hit_points -= 1
            self.check_winner()
        self.player_turn = not self.player_turn
        self.do_effects()
        self.check_winner()
        
    def do_effects(self):
        for name in list(self.effects):
            effect = SPELLS[name]
            self.boss.hit_points -= effect.effect_damage
            self.player.hit_points += effect.heal
            self.player.mana += effect.reward
            self.effects[name] -= 1
            if self.effects[name] == 0:
                self.effects.pop(name)
        
    def player_move(self, spell_name):
        spell = SPELLS[spell_name]
        self.boss.hit_points -= spell.damage
        self.player.hit_points += spell.heal
        if spell.cost > self.player.mana:
            raise OutOfMoney()
        self.player.mana -= spell.cost
        if spell.effect_time:
            if spell.name in self.effects:
                raise EffectActive()
            self.effects[spell.name] = spell.effect_time
        
    def boss_move(self):
        armor = 7 if 'shield' in self.effects else 0
        damage = max(self.boss.damage - armor, 1)
        self.player.hit_points -= damage
        
    def check_winner(self):
        if self.player.hit_points < 1:
            raise BossWins()
        if self.boss.hit_points < 1:
            raise PlayerWins()

In [8]:
def get_next_moves(prev_moves):
    next_moves = set(SPELLS)
    if 'shield' in prev_moves:
        next_moves.discard('shield')
    if 'poison' in prev_moves:
        next_moves.discard('poison')
    if 'recharge' in prev_moves:
        next_moves.discard('recharge')
    yield from next_moves

In [9]:
def play(player_stats, boss_stats, prev_moves, hard=False):
    global best_win
    for next_move in get_next_moves(prev_moves[-3:]):
        game = Game(player_stats, boss_stats, hard)
        for move in prev_moves:
            next(game)
            game.player_move(move)
            next(game)
            game.boss_move()
        next(game)
        moves = prev_moves + [next_move]
        cost = get_cost(moves)
        if best_win and cost >= best_win:
            continue
        try:
            game.player_move(next_move)
            next(game)
            game.boss_move()
            next(game)
        except PlayerWins:
            best_win = cost
            continue
        except (BossWins, OutOfMoney):
            continue
        play(player_stats, boss_stats, moves, hard)

In [10]:
player_stats = (50, 500)
best_win = None
boss_stats = get_boss_stats()
for spell in SPELLS:
    play(player_stats, boss_stats, [spell])

In [11]:
print("Part 1:")
print(best_win)

Part 1:
953


In [12]:
player_stats = (50, 500)
best_win = None
for spell in SPELLS:
    play(player_stats, boss_stats, [spell], hard=True)

In [13]:
print("Part 2:")
print(best_win)

Part 2:
1295
